This Notebook evaluates a convergence study for the Bow Shock Problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_ConvStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_ConvStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_BowShock_ConvStudy'.


{ Session Count = 8; Grid Count = 8; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_ConvStudy }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-04-03 13:44:05Z

In [4]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 3:12:58 PM	13d0abde...
#1: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:20:52 PM	705aac94...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#3: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:16:31 PM	fe883d23...
#4: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:14:05 PM	cab2c050...
#5: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#6: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 3:13:56 PM	643f6a58...
#7: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...


In [5]:
var aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

13d0abde-b5ac-4dfb-9f99-740e96abaeec

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=4$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [8]:
var sessions=database.Sessions.Where(s => (s.Name.Contains("iFlx0") &&s.Name.Contains("iPrb0"))); // the one plotted in the publication
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


#0: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 3:12:58 PM	13d0abde...
#1: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#2: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#3: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...


# A Helper Function to obtain different plot formats

In [9]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Enthalpy Error Plot


In [10]:
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M500.1,57.1 L553.5,57.1 M45.6,110.4 L51.1,111.9 L56.6,113.3 L62.2,114.4 L67.7,116.6 L73.2,121.3
 L78.7,123.4 L84.3,124.4 L89.8,128.6 L95.3,128.7 L100.8,137.0 L106.3,147.5 L111.9,148.7 L117.4,157.1
 L122.9,159.2 L128.4,162.5 L134.0,166.6 L139.5,168.3 L145.0,170.8 L150.5,173.5 L156.1,173.9 L161.6,174.6
 L167.1,175.7 L172.6,179.3 L178.1,181.7 L183.7,181.6 L189.2,181.6 L194.7,181.6 L200.2,181.6 L205.8,181.6
 L211.3,181.6 L216.8,181.6 L222.3,181.6 L227.8,181.6 L233.4,181.6 L238.9,181.6 L244.4,181.6 L249.9,181.6
 L255.5,181.6 L261.0,181.6 L266.5,181.6 L272.0,181.6 L277.5,181.6 L283.1,181.6 L288.6,181.6 L294.1,181.6
 L299.6,181.6 L305.2,181.6 L310.7,181.6 L316.2,181.6 L321.7,181.6 L327.2,181.6 L332.8,181.6 L338.3,181.6
 L343.8,181.6 L349.3,181.6 L354.9,181.6 L360.4,181.6 L365.9,181.6 L371.4,181.6 L377.0,181.6 L382.5,181.6
 L388.0,181.6 L393.5,195.3 L399.0,196.4 L404.6,209.3 L410.1,214.3 L415.6,225.6 L421.1,231.4 L426.7,250.5
 L432.2,261.9 L437.7,279.9 L443.2,235.2 L448.7,262.9 L454.3,278.5 L459.8,281.8 L465.3,353.5 L470.8,362.8
 L476.4,363.3 L481.9,363.0 L487.4,363.1 L492.9,363.2 L498.4,363.2 L504.0,363.3 L509.5,363.3 L515.0,363.3
 L520.5,363.3 L526.1,363.3 L531.6,363.3 L537.1,363.3 L542.6,363.3 L548.1,363.3 L553.7,363.3 L559.2,363.3
 L564.7,363.3 L570.2,363.3 L575.8,363.3 L581.3,363.3 L586.8,363.3 L592.3,363.3 L597.9,363.3 L603.4,363.3
 L608.9,363.3 L614.4,363.3 L619.9,363.3 L625.5,363.3 L631.0,363.3 L636.5,363.3 L642.0,363.3 L647.6,363.3
 L653.1,363.3 L658.6,363.3 L664.1,363.3 L669.6,363.3 L675.2,363.3 L680.7,363.3 L686.2,363.3 L691.7,363.3
 L697.3,363.3 L702.8,363.3 L708.3,363.3 L713.8,363.3 L719.3,363.3 L724.9,363.3 L730.4,363.3 L735.9,363.3
 L741.4,363.3 L747.0,363.3 L752.5,452.4 L758.0,458.0 L763.5,459.9 L769.0,460.6 L774.6,460.6 L780.1,460.6
 L785.6,461.3 L791.1,461.5 L796.7,460.9 L802.2,460.9 L807.7,461.2 L813.2,461.0 L818.8,461.0 L824.3,460.5
 L829.8,460.6 L835.3,461.1 L840.8,460.9 L846.4,460.9 L851.9,460.7 L857.4,460.7 L862.9,460.9 L868.5,460.8
 L874.0,460.8 L879.5,461.0 L885.0,460.9 L890.5,460.9 L896.1,460.8 L901.6,460.8 L907.1,460.9 L912.6,460.9
 L918.2,460.8 L923.7,460.8 L929.2,460.9 L934.7,460.8 L940.2,460.8 L945.8,460.9 L951.3,460.8 L956.8,460.8
 L962.3,460.9 L967.9,460.8 L973.4,460.8 L978.9,460.8 L984.4,460.8 L989.9,460.8 L995.5,460.8 L1001.0,460.8
 L1006.5,460.8 L1012.0,460.8 L1017.6,530.0 L1023.1,530.2 L1028.6,530.1 L1034.1,530.1 L1039.7,530.1 L1045.2,530.2
 L1050.7,530.2 L1056.2,530.1 L1061.7,530.1 L1067.3,530.2 L1072.8,530.2 L1078.3,530.1 L1083.8,530.1 L1089.4,530.1
 L1094.9,530.1 L1100.4,530.1 L1105.9,530.2 L1111.4,530.2 L1117.0,530.2 L1122.5,530.2 L1128.0,530.2 L1133.5,530.2
 L1139.1,530.2 L1144.6,530.2 L1150.1,530.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0

## Residual History

In [11]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1600,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M811.3,57.1 L864.7,57.1 M45.6,86.0 L53.1,87.4 L60.6,87.8 L68.2,88.8 L75.7,90.8 L83.2,93.9
 L90.7,96.6 L98.3,97.7 L105.8,100.0 L113.3,100.7 L120.8,104.0 L128.3,121.4 L135.9,135.4 L143.4,120.4
 L150.9,142.1 L158.4,150.3 L166.0,158.3 L173.5,162.9 L181.0,171.5 L188.5,187.0 L196.0,190.5 L203.6,192.5
 L211.1,197.4 L218.6,187.9 L226.1,194.6 L233.7,211.6 L241.2,224.6 L248.7,222.3 L256.2,228.7 L263.8,230.9
 L271.3,232.0 L278.8,232.9 L286.3,236.5 L293.8,236.8 L301.4,238.8 L308.9,239.1 L316.4,240.4 L323.9,240.3
 L331.5,241.0 L339.0,241.9 L346.5,242.2 L354.0,242.6 L361.5,242.6 L369.1,242.8 L376.6,242.4 L384.1,242.5
 L391.6,242.5 L399.2,242.4 L406.7,243.0 L414.2,243.2 L421.7,243.4 L429.2,243.6 L436.8,243.8 L444.3,243.9
 L451.8,244.1 L459.3,244.2 L466.9,244.2 L474.4,244.3 L481.9,244.3 L489.4,244.3 L496.9,244.4 L504.5,244.3
 L512.0,244.4 L519.5,73.7 L527.0,96.3 L534.6,76.5 L542.1,105.1 L549.6,97.3 L557.1,113.5 L564.7,112.7
 L572.2,126.0 L579.7,142.3 L587.2,174.3 L594.7,226.5 L602.3,270.0 L609.8,276.2 L617.3,281.7 L624.8,282.2
 L632.4,293.4 L639.9,296.6 L647.4,304.0 L654.9,308.1 L662.4,313.1 L670.0,309.6 L677.5,314.2 L685.0,318.9
 L692.5,322.8 L700.1,324.5 L707.6,325.2 L715.1,326.4 L722.6,326.7 L730.1,328.1 L737.7,328.8 L745.2,328.4
 L752.7,329.3 L760.2,329.3 L767.8,329.0 L775.3,329.5 L782.8,329.8 L790.3,330.1 L797.8,330.0 L805.4,330.0
 L812.9,330.2 L820.4,330.6 L827.9,330.6 L835.5,330.7 L843.0,331.0 L850.5,330.8 L858.0,331.2 L865.6,330.8
 L873.1,331.2 L880.6,331.4 L888.1,331.5 L895.6,331.5 L903.2,331.7 L910.7,331.7 L918.2,331.7 L925.7,331.9
 L933.3,331.8 L940.8,331.9 L948.3,331.9 L955.8,332.0 L963.3,331.9 L970.9,332.0 L978.4,332.1 L985.9,332.1
 L993.4,332.1 L1001.0,332.1 L1008.5,282.0 L1016.0,309.7 L1023.5,307.2 L1031.0,310.8 L1038.6,311.2 L1046.1,321.5
 L1053.6,332.8 L1061.1,337.7 L1068.7,334.5 L1076.2,341.1 L1083.7,342.0 L1091.2,347.1 L1098.7,347.9 L1106.3,349.2
 L1113.8,352.3 L1121.3,352.3 L1128.8,355.1 L1136.4,354.9 L1143.9,354.3 L1151.4,355.0 L1158.9,359.9 L1166.5,361.1
 L1174.0,360.3 L1181.5,361.1 L1189.0,361.3 L1196.5,361.5 L1204.1,362.5 L1211.6,363.1 L1219.1,363.0 L1226.6,363.3
 L1234.2,363.8 L1241.7,364.4 L1249.2,365.1 L1256.7,364.8 L1264.2,365.3 L1271.8,365.0 L1279.3,365.7 L1286.8,365.9
 L1294.3,365.5 L1301.9,366.2 L1309.4,366.6 L1316.9,366.5 L1324.4,366.6 L1331.9,366.7 L1339.5,366.7 L1347.0,366.6
 L1354.5,366.7 L1362.0,366.7 L1369.6,342.4 L1377.1,368.0 L1384.6,376.4 L1392.1,386.2 L1399.6,390.0 L1407.2,390.2
 L1414.7,394.6 L1422.2,396.3 L1429.7,396.3 L1437.3,397.9 L1444.8,408.5 L1452.3,408.3 L1459.8,410.4 L1467.4,412.0
 L1474.9,411.8 L1482.4,414.9 L1489.9,415.4 L1497.4,417.1 L1505.0,417.8 L1512.5,416.8 L1520.0,416.9 L1527.5,416.9
 L1535.1,417.2 L1542.6,417.0 L1550.1,417.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGBS-p3-40x128-agg0.4-iPrb0-iFlx

save results to text file

In [12]:
public void SavePlotTable(List<double> Iterations, List<double> y1, string l1, List<double> y2 = null, string l2 = null, List<double> y3 = null, string l3 = null) {
            var table1 = MultidimensionalArray.Create(y1.Count,2);
            table1.SetColumn(0, Iterations);
            table1.SetColumn(1, y1);
            table1.SaveToTextFile(l1 + ".txt");
            if(y2 != null && l2 != null) {
                var table2 = MultidimensionalArray.Create(y2.Count, 2);
                table2.SetColumn(0, Iterations);
                table2.SetColumn(1, y2);
                table2.SaveToTextFile(l2 + ".txt");
            }
            if(y3 != null && l3 != null) {
                var table3 = MultidimensionalArray.Create(y3.Count, 2);
                table3.SetColumn(0, Iterations);
                table3.SetColumn(1, y3);
                table3.SaveToTextFile(l3 + ".txt");
            }
}
var si= sessions.Pick(0);
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
var enthalpyerrors = new List<double>();
foreach(var timestep in si.Timesteps){
    var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
    enthalpyerrors.Add(error);
}
SavePlotTable(lts.TimeStepNumbers,lts.EnResHistory, "EnResNorms", lts.ResHistory, "ResNorms", enthalpyerrors, "EnthalpyErrors")

## Convergence Plot Enthalpy

choose the sessions

they must be reordered for the plot to work...

In [13]:
var ses= sessions;
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

#0: XDGBowShock_TwoLs	XDGBS-p3-5x16-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:05 PM	6ec2412f...
#1: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...
#2: XDGBowShock_TwoLs	XDGBS-p3-20x64-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:43:57 PM	c3bc10a4...
#3: XDGBowShock_TwoLs	XDGBS-p3-40x128-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 3:12:58 PM	13d0abde...


Number of cells

In [14]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

[ 80, 320, 1280, 5120 ]

helper arrays

In [15]:
int[] degs = new int[] {0,1,2,3};
var errors_array= MultidimensionalArray.Create(4,ses.Count());


Function used to calculate the EOCs

In [16]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

get ideal plot

In [17]:
List<double> GetIdealConvPlot(List<double> errors,List<double> cellSize, int p){
        var idealErrors = new List<double>();

    // Calculate ideal errors for each cell size
    for (int i = 0; i < cellSize.Count; i++)
    {
        double idealError = errors[0] * Math.Pow(cellSize[i] / cellSize[0], p + 1);
        idealErrors.Add(idealError);
    }

    return idealErrors;
} 

function for different plot formats

In [18]:
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

do the convergence plot

In [27]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
int count=0;
MultidimensionalArray errorsAndCells=MultidimensionalArray.Create(4,ses.Count()+1);
MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(5,ses.Count()+1);
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        //extract the relevant field 
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            var enthalpyError = (XDGField) lts.GetField("h_err").Clone();
            errors_array[p,iSess]=(enthalpyError.L2NormAllSpecies()/EEN);
            cells.Add(Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0)));
            errors.Add(enthalpyError.L2NormAllSpecies()/EEN);
            Console.WriteLine($"P={p}, h={Math.Sqrt(enthalpyError.Basis.GridDat.iGeomCells.GetCellVolume(0))} ---> error={enthalpyError.L2NormAllSpecies()/EEN}");
        }
    }
    errorsAndCells.SetColumn(0,cells);
    errorsAndCellsIdeal.SetColumn(0,cells);
    errorsAndCells.SetColumn(p+1,errors);
    errorsAndCellsIdeal.SetColumn(p+1,GetIdealConvPlot(errors,cells,p));
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    var fmt = GetFormat(count);
    fmt.PointSize=0.01;
    fmt.DashType=DashTypes.Dashed;

    plot.AddDataGroup($"ideal {p+1}",cells,GetIdealConvPlot(errors,cells,p), fmt);
    count++;
}
errorsAndCells.SaveToTextFile("ConvergenceStudyErrors.txt");
errorsAndCellsIdeal.SaveToTextFile("ConvergenceStudyErrorsIdeal.txt");
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

P=0, h=0.44721359549995787 ---> error=0.023940855986105646
P=0, h=0.22360679774997894 ---> error=0.0134594325396264
P=0, h=0.11180339887498954 ---> error=0.0070032045051262055
P=0, h=0.05590169943749477 ---> error=0.003653535388862935
P=1, h=0.44721359549995787 ---> error=0.0032144940105413734
P=1, h=0.22360679774997894 ---> error=0.0006497499506191623
P=1, h=0.11180339887498954 ---> error=0.00020757010904109995
P=1, h=0.05590169943749477 ---> error=5.869069500295124E-05
P=2, h=0.44721359549995787 ---> error=0.0008750016444413799
P=2, h=0.22360679774997894 ---> error=0.0001259844706090987
P=2, h=0.11180339887498954 ---> error=3.572685002353764E-05
P=2, h=0.05590169943749477 ---> error=6.389593967553816E-06
P=3, h=0.44721359549995787 ---> error=0.00037782501865665476
P=3, h=0.22360679774997894 ---> error=3.481094613539112E-05
P=3, h=0.11180339887498954 ---> error=1.0309590300542182E-05
P=3, h=0.05590169943749477 ---> error=1.3208766585690534E-06
Using gnuplot: C:\Program Files (x86)\FDY

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -8 
 
 
 
 
 10 -7 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 
 ideal 1 
 
 
 ideal 1 
 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 ideal 2 
 
 
 ideal 2 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 ideal 3 
 
 
 ideal 3 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3 
 
 
 
 
 
 
 
 
 
 
 ideal 4 
 
 
 ideal 4

In [28]:
EOCs

[ 0.9040365496187611, 1.9251053533593216, 2.3658058871068572, 2.7200262234005628 ]

## Plotting the ShadowFields (for visualization)

choose a concrete session

In [26]:
var sessions = database.Sessions.Where(s => s.Timesteps.Count >0);
sessions=sessions.Where(s => s.Name.Contains("10x32") && s.Name.Contains("iFlx0"));
sessions

#0: XDGBowShock_TwoLs	XDGBS-p3-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 1:44:06 PM	990fe52f...


plot it

In [25]:
using BoSSS.Solution.Tecplot;
using System.IO;
var si=sessions.Pick(0);
Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));
IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
//double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
var allTs = (List<double>) lts.TimeStepNumbers; 
var N=5;
var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
foreach(var timestep in si.Timesteps){
    if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
            IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
            var SF = lts.GetShadowFields();
            var texplot = new Tecplot(SF[0].GridDat, 2);
            texplot.PlotFields( "BS_SP_" + lts.TimeStepNumber,0.0,SF);
    }
}